In [1]:
import os, sys, math, datetime
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image
import seaborn as sns

import tensorflow as tf
# import tensorflow_datasets as tfds
from tensorflow import keras
# from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D,DepthwiseConv2D, MaxPooling2D, AvgPool2D, GlobalAveragePooling2D, BatchNormalization, Concatenate
# from tensorflow.keras.layers import ReLU
# from tensorflow.keras.models import Model
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
 
# Import the necessary MLTK APIs
from mltk.core import view_model, summarize_model, profile_model

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths, get_file_size
from workbench.utils.utils import parse_model_name, parse_mltk_model_summary
from workbench.data.data import get_lemon_quality_dataset
from workbench.tflite_profiling import get_peak_memory_df, get_tensor_details_df
from workbench.tensorflow import get_layer_details_df

#from dotenv import load_dotenv
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
#import deeplake

%load_ext autoreload



In [2]:
%reload_ext autoreload
%autoreload

In [3]:
models_dir = initialize()

In [4]:
automated = False

global model_name

model_name = "mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1"



In [5]:
base_model_name, alpha, resolution, channels, classes, variation = model_name.split("_")
alpha = float(alpha)
resolution = int(resolution)
classes = int(classes.strip("o"))
channels = int(channels.strip("c"))
PROJECT = "model_DB"

# just needed for INT 8 quantization:
BATCH_SIZE = 32
shuffle_seed = 1

In [6]:
models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

c:\tiny_mlc\tiny_cnn\models


In [7]:
model = keras.models.load_model(models_tf_path)

In [8]:
# view_model(model, tflite=True, build=True)

# Parse model parameters from MLTK summary

In [9]:
total_params, trainable_params, non_trainable_params, MACs, FLOPs = parse_mltk_model_summary(models_summary_path)        

In [10]:
mltk_model_stats = {}

In [11]:
mltk_model_stats["MACs"] = MACs
mltk_model_stats["FLOPs"] = FLOPs
mltk_model_stats["total_params"] = total_params
mltk_model_stats["trainable_params"] = trainable_params
mltk_model_stats["non_trainable_params"] = non_trainable_params

# Creating tflite Models

In [12]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter = tf.lite.TFLiteConverter.from_saved_model(models_path)
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpuk11p_w2\assets


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpuk11p_w2\assets


In [13]:
# Save the model.Y
with open(models_tflite_path, "wb") as f:
    f.write(tflite_model)

In [14]:
mltk_model_stats["model_size_kb"] = get_file_size(models_tf_path)
mltk_model_stats["tflite_model_size_kb"] = get_file_size(models_tflite_path)

File size in bytes is 549400
File size in kilobytes is 536.5234375
File size in bytes is 177216
File size in kilobytes is 173.0625


# tflite with INT8 quantization

In [15]:
dataset_path = Path.cwd().joinpath("datasets", "lemon_dataset")
dataset_path.exists()

True

In [16]:
train_ds, val_ds, test_ds, labels = get_lemon_quality_dataset(dataset_path, resolution, resolution, BATCH_SIZE, channels)

Color mode: rgb
Preparing training dataset...
Found 2021 files belonging to 3 classes.
Preparing validation dataset...
Found 252 files belonging to 3 classes.
Preparing test dataset...
Found 255 files belonging to 3 classes.
Class names: ['bad_quality', 'empty_background', 'good_quality']
Train: (TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
Normalize: True


In [17]:
repr_ds = test_ds.unbatch()

def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(48):
    yield [i_value]
    
converter_opt = tf.lite.TFLiteConverter.from_keras_model(model)

# set the optimization flag
converter_opt.optimizations = [tf.lite.Optimize.DEFAULT]
# enforce integer only quantization
converter_opt.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]


# https://github.com/tensorflow/tensorflow/issues/53293: uint is no longer supported!
#converter_opt.inference_input_type = tf.uint8
#converter_opt.inference_output_type = tf.uint8
converter_opt.inference_input_type = tf.int8
converter_opt.inference_output_type = tf.int8

# provide a representative dataset for quantization
converter_opt.representative_dataset = representative_data_gen

tflite_model_opt = converter_opt.convert()

# Save the model.
with open(models_tflite_opt_path, 'wb') as f:
  f.write(tflite_model_opt)
models_tflite_opt_path

INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpmtxb_k__\assets


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpmtxb_k__\assets
d:\Miniconda\envs\tiny_cnn_6\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


WindowsPath('c:/tiny_mlc/tiny_cnn/models/mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1/mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1_INT8.tflite')

In [18]:
mltk_model_stats["tflite_model_INT8_size_kb"] = get_file_size(models_tflite_opt_path)

File size in bytes is 130000
File size in kilobytes is 126.953125


In [19]:
mltk_model_stats["tflite_model_size_kb"] = get_file_size(models_tflite_path)

File size in bytes is 177216
File size in kilobytes is 173.0625


# Layer Analytics from TensorFlow Model

In [20]:
layer_details_csv_exists = True
try:
    layer_details_df = get_layer_details_df(models_layer_df_path)
    layer_details_df
except:
    layer_details_csv_exists = False

Reading in c:\tiny_mlc\tiny_cnn\models\mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1\mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1_layers.csv


# Getting peak memory infos

In [21]:
models_peak_memory_path = models_dir.joinpath(model_name, f"{model_name}_peak_memory")

In [22]:
#! python i:\tinyml\tflite-tools\tflite_tools.py -i $models_tflite_opt_path --csv $models_peak_memory_path
! python c:\tiny_mlc\tflite-tools\tflite_tools.py -i $models_tflite_opt_path --csv $models_peak_memory_path
#! explorer $models_peak_memory_path

Writing model analysis to c:\tiny_mlc\tiny_cnn\models\mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1\mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1_peak_memory in CSV format


In [23]:
model_peak_memory_schedule = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "execution_schedule_info.csv")
peak_memory_tensor_details = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "tensor_details.csv")

In [24]:
peak_memory_df = get_peak_memory_df(model_peak_memory_schedule)
peak_memory_df.head(10)

Reading in c:\tiny_mlc\tiny_cnn\models\mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1\mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1_peak_memory\execution_schedule_info.csv
Cleaning up the dataframe.


,index,layer_name,tensor_IDs,RAM_b,operator
0,0,block_0_expand_conv2d,"(0, 93)",64512,mobilenetv3smallNSQ/block_0_expand_bn/FusedBat...
1,1,tf.math.multiply_18,"(93, 94)",73728,mobilenetv3smallNSQ/tf.math.multiply_18/Mul
2,2,tf.math.add_18,"(93, 94, 95)",110592,mobilenetv3smallNSQ/tf.math.add_18/Add
3,3,tf.clip_by_value_18,"(93, 95, 96)",110592,mobilenetv3smallNSQ/tf.clip_by_value_18/clip_b...
4,4,tf.clip_by_value_18,"(93, 96, 97)",110592,mobilenetv3smallNSQ/tf.clip_by_value_18/clip_b...
5,5,multiply_18,"(93, 97, 98)",110592,mobilenetv3smallNSQ/multiply_18/mul
6,6,block_1_expand_conv2d,"(98, 99)",73728,mobilenetv3smallNSQ/tf.nn.relu6_6/Relu6;mobile...
7,7,batch_normalization_285,"(99, 100)",73728,mobilenetv3smallNSQ/tf.nn.relu6_7/Relu6;mobile...
8,8,conv2d_95,"(100, 101)",55296,mobilenetv3smallNSQ/batch_normalization_265/Fu...
9,9,block_2_expand_conv2d,"(101, 102)",110592,mobilenetv3smallNSQ/tf.nn.relu6_8/Relu6;mobile...


In [25]:
peak_memory = peak_memory_df["RAM_b"].max()
peak_memory

113320

In [26]:
mltk_model_stats["peak_memory_b"] = peak_memory

In [27]:
peak_memory_df["RAM_b"].argmax()

10

In [28]:
tensor_df = get_tensor_details_df(peak_memory_tensor_details)
tensor_df

Reading in c:\tiny_mlc\tiny_cnn\models\mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1\mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1_peak_memory\tensor_details.csv
Cleaning up the dataframe.


,index,id,layer_name,shape,size_b,name_long
0,0,0,serving_default_input_2:0,"(1, 96, 96, 3)",27648,serving_default_input_2:0
1,1,93,mobilenetv3smallNSQ/block_0_expand_conv2d/Conv2D,"(1, 48, 48, 16)",36864,mobilenetv3smallNSQ/block_0_expand_bn/FusedBat...
2,2,94,mobilenetv3smallNSQ/tf.math.multiply_18/Mul,"(1, 48, 48, 16)",36864,tf.math.multiply_18
3,3,95,mobilenetv3smallNSQ/tf.math.add_18/Add,"(1, 48, 48, 16)",36864,tf.math.add_18
4,4,96,mobilenetv3smallNSQ/tf.clip_by_value_18/clip_b...,"(1, 48, 48, 16)",36864,tf.clip_by_value_18
...,...,...,...,...,...,...
132,132,224,mobilenetv3smallNSQ/multiply_35/mul,"(1, 3, 3, 576)",5184,multiply_35
133,133,225,mobilenetv3smallNSQ/average_pooling2d_1/AvgPool,"(1, 1, 1, 576)",576,average_pooling2d_1
134,134,226,mobilenetv3smallNSQ/flatten_1/Reshape,"(1, 576)",576,flatten_1
135,135,227,mobilenetv3smallNSQ/dense_1/BiasAdd,"(1, 2)",2,mobilenetv3smallNSQ/dense_1/MatMul;mobilenetv3...


# Logging to Weights & Biases

In [29]:
api = wandb.Api()

# Generate run ids
run_id = wandb.sdk.lib.runid.generate_id()

wandb.init(
    project=PROJECT,
    entity="susbrock",
    name = model_name,
    id=run_id,
    #resume=True
    )

config = wandb.config
config.id = run_id
config.update(mltk_model_stats)

# try:
#     layers_df_wandb = wandb.Table(dataframe=layers_df)
#     wandb.log({"tflite_layers_mltk" : layers_df_wandb})
#     layers_INT8_df_wandb = wandb.Table(dataframe=layers_INT8_df)
#     wandb.log({"tflite_INt8_layers_mltk" : layers_INT8_df_wandb})
# except:
#     print("No analysis from mltk!")

peak_memory_df_wandb = wandb.Table(dataframe=peak_memory_df)
wandb.log({"peak_memory" : peak_memory_df_wandb})

tensor_df_wandb = wandb.Table(dataframe=tensor_df)
wandb.log({"tensor_info" : tensor_df_wandb})

if layer_details_csv_exists:
    layer_details_df_wandb = wandb.Table(dataframe=layer_details_df)
    wandb.log({"tensorflow_layer_details" : layer_details_df_wandb})
else:
    pass

try:
    wandb.log({"tflite_INt8_layers_plt" : wandb.plot.bar(layers_INT8_df_wandb, "index", "macs", title=model_name)})
except:
    pass
wandb.finish()



ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: susbrock. Use `wandb login --relogin` to force relogin


# Profiling tflite models with MLTK profiler

In [30]:
mltk_model_stats["architecture"] = model_name
mltk_model_stats["alpha"] = alpha
mltk_model_stats["img_res"] = resolution
mltk_model_stats["classes"] = classes
mltk_model_stats["channels"] = channels
mltk_model_stats["variation"] = variation

In [31]:
# MLTK profile model reads the mode from a path!

profiling = True

try:
    profiling_results = profile_model(str(models_tflite_path), accelerator=None, build=None)
except:
    profiling= False

Profiling model in simulator ...
Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 488832


In [32]:
if automated == False:
    mltk_model_stats["opt_RAM_mltk"] = int(input("Please copy the optimal runtime memory size from above: \n"))

In [33]:
profiling_results.get_summary(exclude_null=False, full_summary=True)
model_profile = profiling_results.to_dict()

In [34]:
mltk_model_stats["flash_model_size_b_mltk"] = model_profile["summary"]["tflite_size"]
mltk_model_stats["RAM_runtime_memory_size_b_mltk"] = model_profile["summary"]["runtime_memory_size"]
mltk_model_stats["OPS_mltk"] = model_profile["summary"]["ops"]
mltk_model_stats["macs_mltk"] = model_profile["summary"]["macs"]
mltk_model_stats["n_unsupported_layers_mltk"] = model_profile["summary"]["n_unsupported_layers"]
mltk_model_stats["energy_mltk"] = model_profile["summary"]["energy"]

In [35]:
layers_df = pd.DataFrame.from_dict(model_profile["layers"])
layers_df["name"] = layers_df["index"].map(str) + ": " + layers_df["opcode"]
layers_df.insert(2,"name" , layers_df.pop("name"))
layers_df

,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,conv_2d,0: conv_2d,2027520,995328,7.353315e+06,1.339589e-03,"1x96x96x3,16x3x3x3,16",1x48x48x16,Padding:same stride:2x2 activation:none
1,1,mul,1: mul,36864,0,0.000000e+00,0.000000e+00,"1x48x48x16,",1x48x48x16,Activation:none
2,2,add,2: add,36864,0,3.102126e+06,6.442078e-04,"1x48x48x16,",1x48x48x16,Activation:none
3,3,minimum,3: minimum,0,0,0.000000e+00,0.000000e+00,"1x48x48x16,",1x48x48x16,Type=none
4,4,relu,4: relu,73728,0,0.000000e+00,0.000000e+00,1x48x48x16,1x48x48x16,Type=none
...,...,...,...,...,...,...,...,...,...,...
131,131,mul,131: mul,5184,0,0.000000e+00,0.000000e+00,"1x3x3x576,1x3x3x576",1x3x3x576,Activation:none
132,132,average_pool_2d,132: average_pool_2d,5760,0,8.169788e+04,4.166768e-06,1x3x3x576,1x1x1x576,Padding:valid stride:1x1 filter:3x3 activation...
133,133,reshape,133: reshape,0,0,2.386067e+02,4.810657e-19,"1x1x1x576,2",1x576,Type=none
134,134,fully_connected,134: fully_connected,2304,1152,3.032674e+03,1.749769e-08,"1x576,2x576",1x2,Activation:none


In [36]:
layers_df_path = pathlib.Path.joinpath(models_dir, "layer_df_mltk.pkl")
layers_df.to_pickle(layers_df_path)

In [37]:
mltk_model_stats

{'MACs': '4.802 M',
 'FLOPs': '9.717 M',
 'total_params': 40490,
 'trainable_params': 37066,
 'non_trainable_params': 3424,
 'model_size_kb': 536.5234375,
 'tflite_model_size_kb': 173.0625,
 'tflite_model_INT8_size_kb': 126.953125,
 'peak_memory_b': 113320,
 'architecture': 'mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1',
 'alpha': 0.1,
 'img_res': 96,
 'classes': 2,
 'channels': 3,
 'variation': 'se4l64.MV1',
 'opt_RAM_mltk': 488832,
 'flash_model_size_b_mltk': 177216,
 'RAM_runtime_memory_size_b_mltk': 482032,
 'OPS_mltk': 10909850,
 'macs_mltk': 4802168,
 'n_unsupported_layers_mltk': 0,
 'energy_mltk': 0.009994241185124153}

In [40]:
# %%capture optimal_runtime_INT
profiling_results_INT8 = profile_model(str(models_tflite_opt_path), accelerator=None, build=False)

Profiling model in simulator ...
Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 150784


In [41]:
if automated == False:    
    mltk_model_stats["opt_RAM_INT8_mltk"] = int(input("Please copy the optimal runtime memory size from above: \n"))

In [42]:
# from contextlib import redirect_stdout

# with open("runtime.txt", "w", encoding='utf-8') as f:
#     with redirect_stdout(f):
#         print(profile_model(str(models_tflite_opt_path), accelerator=None, build=False))

In [43]:
profiling_results_INT8.get_summary()

OrderedDict([('name', 'mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1_INT8'),
             ('accelerator', 'None'),
             ('input_shape', '1x96x96x3'),
             ('input_dtype', 'int8'),
             ('output_shape', '1x2'),
             ('output_dtype', 'int8'),
             ('tflite_size', 130000),
             ('runtime_memory_size', 143404),
             ('ops', 10909850),
             ('macs', 4802168),
             ('n_layers', 136),
             ('n_unsupported_layers', 0),
             ('cpu_cycles', 57964024.5868988),
             ('cpu_utilization', 0.0),
             ('cpu_clock_rate', 78000000),
             ('energy', 0.009994241185124153),
             ('j_per_op', 9.160750317487548e-10),
             ('j_per_mac', 2.08119357446973e-09)])

In [44]:
model_profile_INT8 = profiling_results_INT8.to_dict()


In [45]:
mltk_model_stats["flash_model_size_b__INT8_mltk"] = model_profile_INT8["summary"]["tflite_size"]
mltk_model_stats["RAM_runtime_memory_size_b_INT8_mltk"] = model_profile_INT8["summary"]["runtime_memory_size"]
mltk_model_stats["OPS_INT8_mltk"] = model_profile_INT8["summary"]["ops"]
mltk_model_stats["macs_INT8_mltk"] = model_profile_INT8["summary"]["macs"]
mltk_model_stats["n_unsupported_layers_INT8_mltk"] = model_profile_INT8["summary"]["n_unsupported_layers"]
mltk_model_stats["energy_INT8_mltk"] = model_profile_INT8["summary"]["energy"]

In [46]:
mltk_model_stats

{'MACs': '4.802 M',
 'FLOPs': '9.717 M',
 'total_params': 40490,
 'trainable_params': 37066,
 'non_trainable_params': 3424,
 'model_size_kb': 536.5234375,
 'tflite_model_size_kb': 173.0625,
 'tflite_model_INT8_size_kb': 126.953125,
 'peak_memory_b': 113320,
 'architecture': 'mobilenetv3smallNSQ_0.1_96_c3_o2_se4l64.MV1',
 'alpha': 0.1,
 'img_res': 96,
 'classes': 2,
 'channels': 3,
 'variation': 'se4l64.MV1',
 'opt_RAM_mltk': 488832,
 'flash_model_size_b_mltk': 177216,
 'RAM_runtime_memory_size_b_mltk': 482032,
 'OPS_mltk': 10909850,
 'macs_mltk': 4802168,
 'n_unsupported_layers_mltk': 0,
 'energy_mltk': 0.009994241185124153,
 'opt_RAM_INT8_mltk': 150784,
 'flash_model_size_b__INT8_mltk': 130000,
 'RAM_runtime_memory_size_b_INT8_mltk': 143404,
 'OPS_INT8_mltk': 10909850,
 'macs_INT8_mltk': 4802168,
 'n_unsupported_layers_INT8_mltk': 0,
 'energy_INT8_mltk': 0.009994241185124153}

In [47]:
layers_INT8_df = pd.DataFrame.from_dict(model_profile_INT8["layers"])
layers_INT8_df["name"] = layers_INT8_df["index"].map(str) + ": " + layers_INT8_df["opcode"]
layers_INT8_df.insert(2,"name" , layers_INT8_df.pop("name"))
layers_INT8_df

,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,conv_2d,0: conv_2d,2027520,995328,7.353315e+06,1.339589e-03,"1x96x96x3,16x3x3x3,16",1x48x48x16,Padding:same stride:2x2 activation:none
1,1,mul,1: mul,36864,0,0.000000e+00,0.000000e+00,"1x48x48x16,",1x48x48x16,Activation:none
2,2,add,2: add,36864,0,3.102126e+06,6.442078e-04,"1x48x48x16,",1x48x48x16,Activation:none
3,3,minimum,3: minimum,0,0,0.000000e+00,0.000000e+00,"1x48x48x16,",1x48x48x16,Type=none
4,4,relu,4: relu,73728,0,0.000000e+00,0.000000e+00,1x48x48x16,1x48x48x16,Type=none
...,...,...,...,...,...,...,...,...,...,...
131,131,mul,131: mul,5184,0,0.000000e+00,0.000000e+00,"1x3x3x576,1x3x3x576",1x3x3x576,Activation:none
132,132,average_pool_2d,132: average_pool_2d,5760,0,8.169788e+04,4.166768e-06,1x3x3x576,1x1x1x576,Padding:valid stride:1x1 filter:3x3 activation...
133,133,reshape,133: reshape,0,0,2.386067e+02,4.810657e-19,"1x1x1x576,2",1x576,Type=none
134,134,fully_connected,134: fully_connected,2304,1152,3.032674e+03,1.749769e-08,"1x576,2x576",1x2,Activation:none


In [48]:
layers_df_INT8_path = pathlib.Path.joinpath(models_dir, "layer_df_INT8_mltk.pkl")
layers_INT8_df.to_pickle(layers_df_INT8_path)

In [49]:
#layers_INT8_df.info()

# Getting peak memory infos

# Logging to Weights & Biases - part 2

In [50]:
wandb.init(
    project=PROJECT,
    entity="susbrock",
    name = model_name,
    id=run_id,
    resume="allow"
    )

# config = wandb.config
# config.id = run_id
config.update(mltk_model_stats)

try:
    layers_df_wandb = wandb.Table(dataframe=layers_df)
    wandb.log({"tflite_layers_mltk" : layers_df_wandb})
    layers_INT8_df_wandb = wandb.Table(dataframe=layers_INT8_df)
    wandb.log({"tflite_INt8_layers_mltk" : layers_INT8_df_wandb})
except:
    print("No analysis from mltk!")

# peak_memory_df_wandb = wandb.Table(dataframe=peak_memory_df)
# wandb.log({"peak_memory" : peak_memory_df_wandb})

# tensor_df_wandb = wandb.Table(dataframe=tensor_df)
# wandb.log({"tensor_info" : tensor_df_wandb})

# if layer_details_csv_exists:
#     layer_details_df_wandb = wandb.Table(dataframe=layer_details_df)
#     wandb.log({"tensorflow_layer_details" : layer_details_df_wandb})
# else:
#     pass

# try:
#     wandb.log({"tflite_INt8_layers_plt" : wandb.plot.bar(layers_INT8_df_wandb, "index", "macs", title=model_name)})
# except:
#     pass
wandb.finish()
